In [ ]:
!git clone https://github.com/mehedihasanbijoy/PyTorch-BanglaNLP-Tutorial.git

In [ ]:
# install bnlp toolkit
!pip install -U bnlp_toolkit

In [3]:
# install PySpark and Spark NLP
!pip install -q pyspark==3.1.2 spark-nlp

     |████████████████████████████████| 212.4 MB 51 kB/s 
     |████████████████████████████████| 648 kB 47.4 MB/s 
     |████████████████████████████████| 198 kB 61.6 MB/s 


In [4]:
import pandas as pd
import re
from bnlp import NLTKTokenizer

punkt not found. downloading...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [5]:
df = pd.read_csv('/content/PyTorch-BanglaNLP-Tutorial/0A. Corpus/BanglaEmotion/BanglaEmotion.csv')

## Remove Non-Bangla Characters and Irrelevant Punctuation

In [6]:
# https://en.wikipedia.org/wiki/Bengali_(Unicode_block)

def clean_text(text):
    regex = re.compile('[^ঀ-ৠ ৳,?।]')
    return regex.sub('', text)

## Remove Dublicate Punctuations

In [7]:
punctuations = [',', '?', '।']

def remove_duplicate_punctuations(text):
    _text = [_ for _ in text]

    for i in range(len(text)-1):
        if text[i] in punctuations and text[i+1] in punctuations:
            _text[i] = ''
        elif i < len(text)-2 and text[i] in punctuations and text[i+1] == ' ' and text[i+2] in punctuations:
            _text[i] = ''
    
    return ''.join(_text)

## Bangla Tokenizer

In [8]:
bnltk = NLTKTokenizer()

def tokenizer_bnltk(text):
    return bnltk.word_tokenize(text)

## Bangla Lemmatizer

In [9]:
from pyspark.ml import Pipeline
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

spark = sparknlp.start()

document_assembler = DocumentAssembler().setInputCol("text").setOutputCol("document")
tokenizer = Tokenizer().setInputCols(["document"]).setOutputCol("token")
lemmatizer = LemmatizerModel.pretrained("lemma", "bn").setInputCols(["token"]).setOutputCol("lemma")
nlp_pipeline = Pipeline(stages=[document_assembler, tokenizer, lemmatizer])

light_pipeline = LightPipeline(nlp_pipeline.fit(spark.createDataFrame([[""]]).toDF("text")))

def find_lemma(word):
    lemma = str(light_pipeline.fullAnnotate(word)[0]['lemma'][0]).split('{')[0].split(',')[-2].strip()
    return word, lemma

def lemmatizer(text):
    _text = []
    tokens = tokenizer_bnltk(text) 
    for token in tokens:
        if token in punctuations:
            _text.append(token)
        else:
            token, lemma = find_lemma(token)
            _text.append(lemma)

    return ' '.join(_text)

# lemmatizer("আমাদের মেধা শূন্য এবং বুদ্ধি জীবী শূন্য করা তাদের লক্ষ্য।")

lemma download started this may take some time.
Approximate size to download 90.6 KB
[OK!]


## Convert Sentiment to Label

In [10]:
def sentiment2label(sentiment):
    if sentiment == 'angry':
        return 0
    elif sentiment == 'disgust':
        return 1
    elif sentiment == 'fear':
        return 2
    elif sentiment == 'happy':
        return 3
    elif sentiment == 'sad':
        return 4
    else:
        return 5

In [11]:
df['cleaned_text'] = df['text'].apply(clean_text)
df['cleaned_text'] = df['cleaned_text'].apply(remove_duplicate_punctuations)
# df['cleaned_text'] = df['cleaned_text'].apply(lemmatizer)
df['label'] = df['sentiment'].apply(sentiment2label)

In [12]:
df

,text,sentiment,set,cleaned_text,label
0,"বরাবর, মাননীয় প্রধানমন্ত্রী গণপ্রজাতন্ত্রী বা...",sad,train,"বরাবর, মাননীয় প্রধানমন্ত্রী গণপ্রজাতন্ত্রী বা...",4
1,তদন্ত করে লাভ কী ? কোন দিন কোন তদন্তের পর কিছু...,sad,train,তদন্ত করে লাভ কী ? কোন দিন কোন তদন্তের পর কিছু...,4
2,দেশকি মধ্যম আয়ের দেশে রুপান্তর হচ্ছে নাকি মগের...,angry,train,দেশকি মধ্যম আয়ের দেশে রুপান্তর হচ্ছে নাকি মগের...,0
3,আমি গর্বিত আমি মুসলিম। আমি সংগ্রামী। আমি যোদ্ধ...,happy,train,আমি গর্বিত আমি মুসলিম। আমি সংগ্রামী। আমি যোদ্ধ...,3
4,রাতের বেলা আবার কিসের সকাল।,angry,train,রাতের বেলা আবার কিসের সকাল।,0
...,...,...,...,...,...
5635,ধন্যবাদ ইমারান ভাই আপনার জন্যই এইটা জানতে পারল...,happy,test,ধন্যবাদ ইমারান ভাই আপনার জন্যই এইটা জানতে পারল...,3
5636,বাংলাদেশের সব পুলিশ যুদি এমন হত তাহলে বাংলাদশ ...,happy,test,বাংলাদেশের সব পুলিশ যুদি এমন হত তাহলে বাংলাদশ ...,3
5637,"পলিশ যে জনগনের বন্ধু ,এটা দেখে বুঝা যাচ্ছে।",happy,test,"পলিশ যে জনগনের বন্ধু ,এটা দেখে বুঝা যাচ্ছে।",3
5638,ভাল কর্মের জন্য অশেষ ধন্যবাদ সবাইকে,happy,test,ভাল কর্মের জন্য অশেষ ধন্যবাদ সবাইকে,3


In [13]:
df.to_csv('/content/BanglaEmotion.csv', index=False)